### Modelo de costos: Contratos SAT

> _**Propuesta**_

- Desarrollado por : **Ladino Arturo & Solares Gina** 

#### Secciones

1. Lectura de la información
   * Arquitectura tecnológica MAT - RFC
   * Costos de contratos
2. Preprocesamiento y cálculo de valores
   * Costos trimestrales
   * Porcentaje de contribución
   * Arquitectura y costo
3. Modelos de regresión
   * Mínimos cuadrados (OLS)
   * Mínimos cuadrados parciales (PLS)
4. Resultados

----

In [1]:
#%#%#%#%#%#%#%# Librerias #%#%#%#%#%#%#%#

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore",
                        category = DeprecationWarning)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


* **Lectura de la información**

In [2]:
#%#%#%#%#%#%#%# Lectura de la información #%#%#%#%#%#%#%#

Entrada = '/Users/ladino/Desktop/SAT_COSTOS/'

#%#%#%#%#%#%#%# Base de datos #%#%#%#%#%#%#%#

COSTO = pd.read_excel(Entrada + 'Base_Uso_costos.xlsx')
MATRFC = pd.read_excel(Entrada + 'RFCMAT.xlsx')


In [3]:
#%#%#%#%#%#%#%# Función renombramiento columnas [F00] #%#%#%#%#%#%#%#

def Rename_SAT(Data, Pre, Letra):
    # Generar nuevos nombres de columnas
    Name = ['{}{}'.format(Pre, Letra.format(i+1)) for i in range(len(Data.columns))]
    # Guardar los nombres originales y nuevos en un diccionario
    Dict = dict(zip(Data.columns, Name))
    # Cambiar el nombre de todas las columnas por los nuevos nombres
    Data = Data.rename(columns=Dict)
    return Dict, Data

In [4]:
#%#%#%#%#%#%#%# Aplicación de funcioón: F00 #%#%#%#%#%#%#%#

Costo_dict, Costo_name = Rename_SAT(COSTO, 'C', '{}')
Mtrfc_dict, Mtrfc_name = Rename_SAT(MATRFC, 'M', '{}')

* **Preprocesamiento y cálculo de valores**

In [5]:
#%#%#%#%#%#%#%# Diccionario de costos
#%# ID contrato y Grupo de Servicio
Costo_dict

{'Contrato': 'C1',
 'ID contrato': 'C2',
 'ID servicio': 'C3',
 'Grupo de Servicio': 'C4',
 'Concepto de Servicio': 'C5',
 'Precio Unitario': 'C6',
 'Volumetría Mínima': 'C7',
 'Volumetría Máxima': 'C8',
 'Volumetría Último CM': 'C9',
 'Unidades enero': 'C10',
 'Monto enero': 'C11',
 'Unidades febrero': 'C12',
 'Monto febrero': 'C13',
 'Unidades marzo': 'C14',
 'Monto marzo': 'C15',
 'Unidades abril': 'C16',
 'Monto abril': 'C17',
 'Unidades mayo': 'C18',
 'Monto mayo': 'C19',
 'Unidades junio': 'C20',
 'Monto junio': 'C21'}

In [6]:
#%#%#%#%#%#%#%# Costos trimestrales #%#%#%#%#%#%#%#

# Se agrupan los datos por las columnas C2 y C4, y luego se suman las columnas específicas
Costo_trime = Costo_name.groupby(['C2',
                                  'C4']).agg({'C11':'sum', 
                                              'C13':'sum', 
                                              'C15':'sum',
                                              'C17':'sum', 
                                              'C19':'sum', 
                                              'C21':'sum'}).reset_index()

# Se calcula el total del trimestre 1 sumando las columnas C11, C13 y C15
Costo_trime['Trimestre1'] = Costo_trime['C11'] + Costo_trime['C13'] + Costo_trime['C15']

# Se calcula el total del trimestre 2 sumando las columnas C17, C19 y C21
Costo_trime['Trimestre2'] = Costo_trime['C17'] + Costo_trime['C19'] + Costo_trime['C21']

# Se crea la tabla de uso con las columnas 'C2', 'C4', 'Trimestre1' y 'Trimestre2'
Trimestral = Costo_trime[['C2', 'C4','Trimestre1','Trimestre2']]


Trimestral.head(5)

,C2,C4,Trimestre1,Trimestre2
0,IBM 5,Unica,0.00,0.00
1,ISITI,SERVICIOS AGREGADOS. PTOS INFRAEST ADMIN (SOPO...,61868307.70,61659215.38
2,ISITI,Servicios Desagregados,13846633.38,688609.80
3,MDR IV,USO DE PATENTES Y MARCAS,0.00,0.00
4,Microsoft 4 Segundo Contrato,USO DE PATENTES Y MARCAS,0.00,0.00


> **Porcentaje de contribución**

$$ \% = \frac{C4_{i}}{C2_j}* 100 $$

* $C4_{i}$ = Costo por grupo de servicio $i$
* $C2_{j}$ = Costo total por servicio $j$

In [45]:
#%#%#%#%#%#%#%# Porcentaje de contribución por Contrato y Grupo de Servicio #%#%#%#%#%#%#%#

# Se calcula la suma vertical de la columna 'Trimestre1' por cada clase en C2
suma_trimestre1_por_clase = Trimestral.groupby('C2')['Trimestre1'].transform('sum')

# Se calcula la suma vertical de la columna 'Trimestre2' por cada clase en C2
suma_trimestre2_por_clase = Trimestral.groupby('C2')['Trimestre2'].transform('sum')

# Se calcula el porcentaje de contribución de cada subclase en C4 en el trimestre1 por cada clase en C2
Trimestral.loc[:, 'PorTimestre1'] = round((Trimestral['Trimestre1'] / suma_trimestre1_por_clase) , 3)

# Calcular el porcentaje de contribución de cada subclase en C4 en el trimestre2 por cada clase en C2
Trimestral.loc[:, 'PorTimestre2'] = round((Trimestral['Trimestre2'] / suma_trimestre2_por_clase) , 3)

# Muestra para validar que se cumple la forma esperada
Trimestral.query('C2 == "SASIC 2"')

,C2,C4,Trimestre1,Trimestre2,PorTimestre1,PorTimestre2
12,SASIC 2,Ciberseguridad,3257201.82,3257201.82,0.036,0.045
13,SASIC 2,Criptografía,7351522.71,10781522.71,0.082,0.150
14,SASIC 2,Desagregados,149640.96,345349.96,0.002,0.005
15,SASIC 2,Infraestructura,22383117.42,21755175.22,0.249,0.302
16,SASIC 2,Protección,14363827.70,14589198.40,0.160,0.203
17,SASIC 2,Seguridad de la Información,42298469.30,21207052.43,0.471,0.295
18,SASIC 2,Transición,0.00,0.00,0.000,0.000


In [52]:
# Validación del porcentaje de contribución por ID Contrado con base a Grupo de Servicio
# Trimeste 01

print (Trimestral.groupby(['C2'])['PorTimestre1'].sum())

C2
IBM 5                           0.000
ISITI                           1.000
MDR IV                          0.000
Microsoft 4 Segundo Contrato    0.000
Microsoft 5                     1.000
SAC 3 P1                        1.000
SAC 3 P3                        0.000
SADCON 6 Segundo Contrato       1.000
SADCON 7                        0.000
SARI 2 P1                       1.000
SARI 2 P2                       1.000
SASIC 2                         1.000
SATFM                           0.999
SCD 6                           1.000
SDMA 6                          1.000
SENHA                           1.000
SENHA 1.1                       1.000
SIDyF 5                         1.001
SPI-2                           1.000
SSO 4                           1.000
STLD 4                          1.000
Name: PorTimestre1, dtype: float64


In [53]:
# Validación del porcentaje de contribución por ID Contrado con base a Grupo de Servicio
# Trimeste 02

print (Trimestral.groupby(['C2'])['PorTimestre2'].sum())

C2
IBM 5                           0.000
ISITI                           1.000
MDR IV                          0.000
Microsoft 4 Segundo Contrato    0.000
Microsoft 5                     0.000
SAC 3 P1                        1.000
SAC 3 P3                        0.000
SADCON 6 Segundo Contrato       1.000
SADCON 7                        1.000
SARI 2 P1                       1.000
SARI 2 P2                       1.000
SASIC 2                         1.000
SATFM                           1.000
SCD 6                           1.000
SDMA 6                          1.000
SENHA                           1.000
SENHA 1.1                       1.000
SIDyF 5                         0.999
SPI-2                           1.000
SSO 4                           0.000
STLD 4                          1.000
Name: PorTimestre2, dtype: float64


In [9]:
#%#%#%#%#%#%#%# Arquitectura tecnológica MAT-RFC #%#%#%#%#%#%#%#

MATRFC.head(5)

,Dimension,Origen,Puerto,Servidor,Seccion
0,x1,LAG Empleado,10.56.24.88 Puerto: 443,tridnprolaga01 10.56.40.4,Acceso y Presentacion
1,x1,LAG Empleado,10.56.24.88 Puerto: 443,tridnprolaga02 10.56.40.10,Acceso y Presentacion
2,x1,LAG Empleado,10.56.24.88 Puerto: 443,tridnprolaga03 10.56.40.16,Acceso y Presentacion
3,x1,LAG Empleado,10.56.24.88 Puerto: 443,tridnprolaga04 10.56.40.22,Acceso y Presentacion
4,x2,Servidor HTTP,10.56.8.14 Puerto: 9080,tq12cproohse01 10.56.78.44,Acceso y Presentacion


In [54]:
#%#%#%#%#%#%#%# Porcentaje de contribución por Dimensión, Puerto y Servidor #%#%#%#%#%#%#%#

# Se agrupan los datos por las columnas 'Dimension' y 'Puerto' para poder
# contar el número de servidores para cada combinación
Servidores = MATRFC.groupby(['Dimension','Puerto'])['Servidor'].count().reset_index()

# Se calcula la proporción de servidores para cada combinación única de 'Dimension' y 'Puerto'
Servidores['PorServidor'] = (Servidores.Servidor / Servidores.Servidor.sum())

# Muestra de la información
Servidores.head(4)

,Dimension,Puerto,Servidor,PorServidor
0,x1,10.56.24.88 Puerto: 443,4,0.014493
1,x10,IPSCAN 10.56.246.133,15,0.054348
2,x10,IPSCAN 10.56.246.134,15,0.054348
3,x10,IPSCAN 10.56.246.135,15,0.054348


In [55]:
# Se agrupan los datos en el DataFrame 'Servidores' por la columna 'Dimension'
# y se calcula la suma de los valores en las columnas 'Servidor' y 'PorServidor' dentro de cada grupo
# con esto sabemos el porcentaje de cada modulo "X" para el modelo

Servidor_xs = Servidores.groupby(['Dimension']).agg({'Servidor':'sum',
                                                     'PorServidor':'sum'}).reset_index()
Servidor_xs.head(5)

,Dimension,Servidor,PorServidor
0,x1,4,0.014493
1,x10,45,0.163043
2,x11,45,0.163043
3,x12,12,0.043478
4,x13,18,0.065217


In [58]:
# Validación del porcentaje de contribución por Servidor con base a Dimensión

print (Servidor_xs.PorServidor.sum())

1.0


> En este punto tenemos:

1. Porcentaje de contribución por grupo de servicio
2. Porcentaje de contribución por servidor en los bloques de dimensión

In [59]:
#%#%#%#%#%#%#%# Arquitectura y Costos #%#%#%#%#%#%#%#

#### Hipótesis propuesta 

* El valor del costo con respecto a la arquitectua y al grupo de servicios la podemos definir de la siguiente forma:

$$ VC_{sub} = \sum_{i,j=1}^{n} \frac{C_{i}}{Ar}$$

* Donde $C_{i}$ es el costo por grupo de servicio con $P_{i}$ porcentaje de contribución ; $Ar$ total de los elementos de la arquitectura



In [80]:
# Total de servidores

print(Servidor_xs.Servidor.sum())

276


In [81]:
# Se hace una copia del DataFrame 'Trimestral' para trabajar con él sin modificar el original
Valor_Costo = Trimestral.copy()

# Se calcula el costo promedio por servidor para el Trimestre 1 y asignarlo a la columna 'CPi_T1'
Valor_Costo['CPi_T1'] =  Valor_Costo.Trimestre1 / Servidor_xs.Servidor.sum()

# Se calcula  el costo promedio por servidor para el Trimestre 2 y asignarlo a la columna 'CPi_T2'
Valor_Costo['CPi_T2'] =  Valor_Costo.Trimestre2 / Servidor_xs.Servidor.sum()

# Muestra de la información
Valor_Costo.head(3)

,C2,C4,Trimestre1,Trimestre2,PorTimestre1,PorTimestre2,CPi_T1,CPi_T2
0,IBM 5,Unica,0.00,0.00,NaN,NaN,0.000000,0.000000
1,ISITI,SERVICIOS AGREGADOS. PTOS INFRAEST ADMIN (SOPO...,61868307.70,61659215.38,0.817,0.989,224160.535145,223402.954275
2,ISITI,Servicios Desagregados,13846633.38,688609.80,0.183,0.011,50168.961522,2494.963043


Entonces para cada sub se tiene un porcentaje de atribucion por arquitectura, falta ver como se monta el porcentaje en las variables X con respecto a la arquitectua 

In [71]:
#%#%#%#%#%#%#%# Total por contrato #%#%#%#%#%#%#%#

# Columnas que se sumarán horizontalmente por cada grupo
Col_Costos = ['C11', 'C13', 'C15', 'C17', 'C19', 'C21']

# Se agrupa el DataFrame por la columna 'C2', 
# Se sleccionan las columnas especificadas y se suman los valores dentro de cada grupo
# Se transpone el resultado para que la suma se realice horizontalmente en lugar de verticalmente
# Se suma horizontalmente los valores resultantes de cada grupo, para dar como resultado el valor por clase

Costo_Clase = Costo_name.groupby(['C2'])[Col_Costos].sum().\
              transpose().sum().reset_index().\
              rename({0: 'Total'}, axis = 1)
Costo_Clase.head(3)

,C2,Total
0,IBM 5,0.000000e+00
1,ISITI,1.380628e+08
2,MDR IV,0.000000e+00
